In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Schizophrenia"
cohort = "GSE161986"

# Input paths
in_trait_dir = "../../input/GEO/Schizophrenia"
in_cohort_dir = "../../input/GEO/Schizophrenia/GSE161986"

# Output paths
out_data_file = "../../output/preprocess/Schizophrenia/GSE161986.csv"
out_gene_data_file = "../../output/preprocess/Schizophrenia/gene_data/GSE161986.csv"
out_clinical_data_file = "../../output/preprocess/Schizophrenia/clinical_data/GSE161986.csv"
json_path = "../../output/preprocess/Schizophrenia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Network preservation reveals shared and unique biological processes associated with chronic alcohol abuse in the NAc and PFC [mRNA]"
!Series_summary	"Chronic alcohol abuse has been linked to the disruption of executive function and allostatic conditioning of reward response dysregulation in the mesocorticolimbic pathway (MCL). Here, we analyzed genome-wide mRNA and miRNA expression from matched cases with alcohol dependence (AD) and controls (n=35) via gene network analysis to identify unique and shared biological processes dysregulated in the prefrontal cortex (PFC) and nucleus accumbens (NAc). We further investigated potential mRNA/miRNA interactions at the network and individual gene expression levels to identify the neurobiological mechanisms underlying AD in the brain. By using genotyped and imputed SNP data, we identified expression quantitative trait loci (eQTL) uncovering potential genetic regulatory elements for gene networks associated w

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the title and summary, we can see this dataset contains mRNA data, which is gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Schizophrenia) - The dataset is about Alcohol Dependence, not Schizophrenia
# Looking at key 1: 'diagnosis: Alcohol', 'diagnosis: Control' - this contains alcohol diagnosis information
trait_row = 1

# Age - Looking at key 2, there are age values for different samples
age_row = 2

# Gender - Looking at key 3, all samples are male (constant value)
# Since constant features are useless, gender is considered unavailable
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert the trait values to binary values."""
    if value is None:
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary where Alcohol is 1 (case) and Control is 0 (control)
    if "alcohol" in value.lower():
        return 1
    elif "control" in value.lower():
        return 0
    return None

def convert_age(value):
    """Convert the age values to continuous values."""
    if value is None:
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# No need for convert_gender as gender_row is None, but we'll define it anyway for completeness
def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.lower() == "male":
        return 1
    elif value.lower() == "female":
        return 0
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Assuming clinical_data was already loaded and available from previous step
    # Extract clinical features using the provided function
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data, 
        trait=trait, 
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM4929029': [1.0, 61.0], 'GSM4929030': [0.0, 44.0], 'GSM4929031': [0.0, 62.0], 'GSM4929032': [1.0, 56.0], 'GSM4929033': [0.0, 63.0], 'GSM4929034': [1.0, 42.0], 'GSM4929035': [0.0, 46.0], 'GSM4929036': [0.0, 56.0], 'GSM4929037': [1.0, 52.0], 'GSM4929038': [0.0, 43.0], 'GSM4929039': [1.0, 59.0], 'GSM4929040': [1.0, 56.0], 'GSM4929041': [1.0, 54.0], 'GSM4929042': [1.0, 46.0], 'GSM4929043': [1.0, 39.0], 'GSM4929044': [1.0, 73.0], 'GSM4929045': [0.0, 56.0], 'GSM4929046': [0.0, 50.0], 'GSM4929047': [1.0, 63.0], 'GSM4929048': [1.0, 50.0], 'GSM4929049': [1.0, 50.0], 'GSM4929050': [1.0, 51.0], 'GSM4929051': [1.0, 64.0], 'GSM4929052': [1.0, 55.0], 'GSM4929053': [0.0, 55.0], 'GSM4929054': [0.0, 47.0], 'GSM4929055': [0.0, 50.0], 'GSM4929056': [0.0, 55.0], 'GSM4929057': [1.0, 53.0], 'GSM4929058': [0.0, 82.0], 'GSM4929059': [0.0, 64.0], 'GSM4929060': [1.0, 73.0], 'GSM4929061': [0.0, 73.0], 'GSM4929062': [0.0, 57.0], 'GSM4929063': [0.0, 59.0]}
Clinical data saved to .

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Schizophrenia/GSE161986/GSE161986_series_matrix.txt.gz
Gene data shape: (22215, 35)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers provided in the output
# The identifiers like '1007_s_at', '1053_at', etc. appear to be Affymetrix probe IDs
# These are not standard human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

Column 'GB_ACC' contains values matching gene data ID pattern


Column 'Target Description' contains values matching gene data ID pattern
Column 'Representative Public ID' contains values matching gene data ID pattern


Column 'Gene Title' contains values matching gene data ID pattern


Column 'ENTREZ_GENE_ID' contains values matching gene data ID pattern
Column 'RefSeq Transcript ID' contains values matching gene data ID pattern


Column 'Gene Ontology Biological Process' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Species Scientific Name' may contain gene-related information
Sample values: ['Homo sapiens', 'Homo sapiens', 'Homo sapiens']
Column 'Target Description' may contain gene-related information
Sample values: ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene"]
Column 'Gene Title' may contain gene-related information
Sample values: ['discoidin domain receptor tyrosine kinase 1 /// microRNA 4640', 'replication factor C (activator 1) 2, 40kDa', "heat shock 70kDa protein 6 (HSP70B')"]
Column 'Gene Symbol' may contain gene-related information
Sample values: ['DDR1 /// MIR4640', 'RFC2', 'HSPA6']
Column 'ENTREZ_GENE_ID' may contain gene-related information
Sample values: ['780 /// 100616237

### Step 6: Gene Identifier Mapping

In [7]:
# Check the mapping information from the gene annotation data
# From the previous analysis, I see that 'ID' column matches the gene data identifiers and 'Gene Symbol' column contains gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Sample of gene mapping data:")
print(preview_df(gene_mapping, n=5))

# Apply the gene mapping, but add a progress tracker
print("Starting gene mapping process...")

# Let's examine the gene symbols to see if many probes map to multiple genes
gene_mapping['num_genes'] = gene_mapping['Gene'].apply(lambda x: len(extract_human_gene_symbols(x)) if isinstance(x, str) else 0)
multi_gene_count = (gene_mapping['num_genes'] > 1).sum()
print(f"Number of probes mapping to multiple genes: {multi_gene_count} ({multi_gene_count/len(gene_mapping)*100:.2f}%)")

# Optimize by only processing probes that exist in gene_data
gene_mapping = gene_mapping[gene_mapping['ID'].astype(str).isin(gene_data.index)]
print(f"Reduced mapping dataframe to only probes in gene_data: {gene_mapping.shape}")

# Apply the gene mapping with optimized memory usage
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted gene expression data shape: {gene_data.shape}")
print("First few genes and their values:")
print(gene_data.head(5))

# Normalize gene symbols using the built-in function
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene expression data shape: {gene_data.shape}")
print("First few normalized genes:")
print(gene_data.index[:10].tolist())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (21225, 2)
Sample of gene mapping data:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}
Starting gene mapping process...
Number of probes mapping to multiple genes: 1067 (5.03%)
Reduced mapping dataframe to only probes in gene_data: (21215, 3)


Converted gene expression data shape: (13830, 35)
First few genes and their values:
        GSM4929029  GSM4929030  GSM4929031  GSM4929032  GSM4929033  \
Gene                                                                 
A1CF       4.54833     5.02891     4.50991     4.59167     4.48148   
A2M        9.18608     8.91580     8.58664     9.40771     8.99435   
A4GALT     6.48012     6.39153     6.33478     6.48012     6.28686   
A4GNT      4.38932     4.20068     4.50872     4.37435     4.39409   
AAAS       6.24990     6.20668     6.32526     5.91476     6.21621   

        GSM4929034  GSM4929035  GSM4929036  GSM4929037  GSM4929038  ...  \
Gene                                                                ...   
A1CF       4.45979     4.74628     4.87043     4.78021     5.00795  ...   
A2M        7.56797     8.93715     9.13947    10.30000     9.41105  ...   
A4GALT     6.67685     6.67250     6.50615     6.48012     6.86503  ...   
A4GNT      4.35462     4.37792     4.46890     4.4

After normalization, gene expression data shape: (13542, 35)
First few normalized genes:
['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB', 'AAK1', 'AAMDC']


Gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE161986.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data - already done in previous step
print(f"Gene data shape after normalization: {gene_data.shape}")

# Save the normalized gene data to file - already done in previous step
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Load the clinical features from the previous step
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(selected_clinical_df.head())

# Link the clinical and genetic data directly using the properly encoded clinical data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data related to alcohol dependence but was evaluated for Schizophrenia."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")

Gene data shape after normalization: (13542, 35)
Normalized gene expression data saved to ../../output/preprocess/Schizophrenia/gene_data/GSE161986.csv
Selected clinical data shape: (2, 35)
Clinical data preview:
               GSM4929029  GSM4929030  GSM4929031  GSM4929032  GSM4929033  \
Schizophrenia         1.0         0.0         0.0         1.0         0.0   
Age                  61.0        44.0        62.0        56.0        63.0   

               GSM4929034  GSM4929035  GSM4929036  GSM4929037  GSM4929038  \
Schizophrenia         1.0         0.0         0.0         1.0         0.0   
Age                  42.0        46.0        56.0        52.0        43.0   

               ...  GSM4929054  GSM4929055  GSM4929056  GSM4929057  \
Schizophrenia  ...         0.0         0.0         0.0         1.0   
Age            ...        47.0        50.0        55.0        53.0   

               GSM4929058  GSM4929059  GSM4929060  GSM4929061  GSM4929062  \
Schizophrenia         0.0         0

Data shape after handling missing values: (35, 13544)
For the feature 'Schizophrenia', the least common label is '0.0' with 17 occurrences. This represents 48.57% of the dataset.
The distribution of the feature 'Schizophrenia' in this dataset is fine.

Quartiles for 'Age':
  25%: 50.0
  50% (Median): 55.0
  75%: 61.5
Min: 39.0
Max: 82.0
The distribution of the feature 'Age' in this dataset is fine.

Data shape after removing biased features: (35, 13544)


Linked data saved to ../../output/preprocess/Schizophrenia/GSE161986.csv
